In [63]:
from skyfield.api import load, wgs84
import numpy as np
from datetime import datetime, timedelta

stations_url = 'http://celestrak.org/NORAD/elements/stations.txt'
satellites = load.tle_file(stations_url)
print('Loaded', len(satellites), 'satellites')

Loaded 29 satellites


In [54]:
by_name = {sat.name: sat for sat in satellites}
satellite = by_name['ISS (ZARYA)']
print(satellite)

ISS (ZARYA) catalog #25544 epoch 2023-06-25 04:29:30 UTC


In [55]:
from geopy import Nominatim
try : 
    geolocator = Nominatim(user_agent="MyApp")
    location = geolocator.geocode('Old Subhash Nagar , Bhopal')
    lat = round(float(location.latitude), 5)
    lon = round(float(location.longitude), 5)
    print(lat , lon)

except Exception as e : 
    print('An error occured while retrieving the location : ',e)
     

23.25009 77.42922


In [ ]:
 
bpl = wgs84.latlon(23.25009, 77.42922)
ts = load.timescale()
t = ts.now()
startTime= ts.utc(2023, 6, 25)
EndTime = ts.utc(2023, 6, 27)
t , events  = satellite.find_events(bpl, startTime, EndTime)
event_names = 'AOS', 'MAX', 'LOS'
 
for ti, event in zip(t, events):
    name = event_names[event]
    print(ti.utc_strftime('%Y %b %d %H:%M:%S') , name )

In [27]:
t = ts.now()
geocentric = satellite.at(t)
print(geocentric.position.km)

[-4240.91423113  3902.58765103 -3608.86099781]


In [28]:
lat, lon = wgs84.latlon_of(geocentric)
print('Latitude:', lat)
print('Longitude:', lon)

Latitude: -32deg 18' 39.5"
Longitude: 77deg 01' 39.4"


In [29]:
difference = satellite - bpl
topocentric = difference.at(t)
print(topocentric.position.km)

[   91.73400489   -41.64755937 -6120.75346444]


In [32]:
alt, az, distance = topocentric.altaz()

if alt.degrees > 0:
    print('The ISS is above the horizon')

print('Altitude:', alt.degrees)
print('Azimuth:', az.degrees)
print('Distance: {:.1f} km'.format(distance.km))

Altitude: -24.241972613298874
Azimuth: 180.41423403241703
Distance: 6121.6 km


In [56]:
bpl = wgs84.latlon(23.25009, 77.42922)
ts = load.timescale()
t = ts.now()

startTime= ts.utc(2023, 6, 25)
EndTime = ts.utc(2023, 6, 26)

difference = satellite - bpl

t , events  = satellite.find_events(bpl, startTime, EndTime , altitude_degrees=0)
event_names = 'AOS', 'MAX', 'LOS'

for ti,event in zip(t,events):
    timePath = ti 
    topocentric = difference.at(timePath)
    alt, az, distance = topocentric.altaz()
    name = event_names[event]
    print(name , ti.utc_strftime('%d-%b-%y %H:%M:%S') ,np.round(alt.degrees , 2) , np.round(az.degrees,2) ) 
    if name == 'LOS':
        print()

AOS 25-Jun-23 04:44:04 0.0 358.1
MAX 25-Jun-23 04:47:22 4.41 36.28
LOS 25-Jun-23 04:50:40 -0.0 74.45

AOS 25-Jun-23 06:19:08 0.0 322.39
MAX 25-Jun-23 06:24:32 68.7 49.43
LOS 25-Jun-23 06:29:55 -0.0 136.08

AOS 25-Jun-23 07:57:33 0.0 275.35
MAX 25-Jun-23 08:00:48 4.46 237.75
LOS 25-Jun-23 08:04:03 -0.0 199.98

AOS 25-Jun-23 19:42:37 0.01 175.89
MAX 25-Jun-23 19:46:51 9.66 123.59
LOS 25-Jun-23 19:51:04 -0.0 71.47

AOS 25-Jun-23 21:17:58 0.0 234.85
MAX 25-Jun-23 21:23:14 36.14 312.87
LOS 25-Jun-23 21:28:31 -0.0 30.97

AOS 25-Jun-23 22:58:32 0.0 303.47
MAX 25-Jun-23 23:00:39 1.56 326.98
LOS 25-Jun-23 23:02:46 -0.0 350.49



In [78]:
#get accurate date per time 
bpl = wgs84.latlon(23.25009, 77.42922)
ts = load.timescale()
t = ts.now()

 
steps = 60 #seconds

predictionStartTime= ts.utc(2023, 6, 25)
predictionEndTime = ts.utc(2023, 6, 26)

difference = satellite - bpl

t , events  = satellite.find_events(bpl, predictionStartTime, predictionEndTime , altitude_degrees=0)
event_names = 'AOS', 'MAX', 'LOS'

startTime = None 
endTime = None

 

for idx, (ti, event) in enumerate(zip(t, events)):
    timePath = ti
    topocentric = difference.at(timePath)
    alt, az, distance = topocentric.altaz()

    name = event_names[event]
   
    print("Object Format:", (ti, event))
    print(name, ti.utc_strftime('%d-%b-%y %H:%M:%S'), np.round(alt.degrees, 2), np.round(az.degrees, 2))

    if event == 0 and startTime is None: 
        startTime = ti.utc_strftime("%Y-%m-%d %H:%M:%S") 
    if event == 2 and endTime is None: 
        endTime = ti.utc_strftime("%Y-%m-%d %H:%M:%S")

    if startTime is not None and endTime is not None:
        break


Object Format: (<Time tt=2460120.6980687953>, 0)
AOS 25-Jun-23 04:44:04 0.0 358.1
Object Format: (<Time tt=2460120.7003604006>, 1)
MAX 25-Jun-23 04:47:22 4.41 36.28
Object Format: (<Time tt=2460120.7026500017>, 2)
LOS 25-Jun-23 04:50:40 -0.0 74.45


In [61]:
from datetime import datetime, timedelta

start_time = datetime(2023, 6, 25, 9, 0, 0)  # Example start time: June 25, 2023, 9:00:00
end_time = datetime(2023, 6, 25, 10, 0, 0)  # Example end time: June 25, 2023, 18:00:00
 
time_interval = timedelta(minutes=10)
 
current_time = start_time
 
while current_time <= end_time: 
    print(current_time.strftime("%Y-%m-%d %H:%M:%S"))
    current_time += time_interval


2023-06-25 09:00:00
2023-06-25 09:10:00
2023-06-25 09:20:00
2023-06-25 09:30:00
2023-06-25 09:40:00
2023-06-25 09:50:00
2023-06-25 10:00:00


In [81]:
print(startTime.strftime("%Y-%m-%d %H:%M:%S") , endTime.strftime("%Y-%m-%d %H:%M:%S"))

AttributeError: 'Time' object has no attribute 'strftime'